#### Imports

In [ ]:
from fastapi import FastAPI, HTTPException
from typing import List, Dict
import requests
import uvicorn


# Load environment variables from .env file
load_dotenv()

# Create the FastAPI app
app = FastAPI()

#### API KEYS

In [2]:
# List of API keys (loaded from environment variables)
API_KEYS = [
    os.getenv("API_KEY_1"),
    os.getenv("API_KEY_2"),
    os.getenv("API_KEY_3"),
    os.getenv("API_KEY_4"),
    os.getenv("API_KEY_5"),
    os.getenv("API_KEY_6"),
    os.getenv("API_KEY_7"),
    os.getenv("API_KEY_8"),
    os.getenv("API_KEY_9"),
    os.getenv("API_KEY_10"),
    os.getenv("API_KEY_11"),
    os.getenv("API_KEY_12"),
]

#### Quota Limit

In [3]:
# Quota limit per API key
QUOTA_LIMIT = 10000

In [ ]:
# Dictionary to track quota usage
quota_usage = {key: 0 for key in API_KEYS}

# Index to keep track of the current API key
current_key_index = 0

def get_next_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(API_KEYS)
    return API_KEYS[current_key_index]

def make_youtube_api_request(url: str, params: Dict):
    global quota_usage

    while True:
        current_key = API_KEYS[current_key_index]
        if quota_usage[current_key] >= QUOTA_LIMIT:
            get_next_api_key()
            continue

        params['key'] = current_key
        response = requests.get(url, params=params)

        if response.status_code == 403 and 'quotaExceeded' in response.text:
            quota_usage[current_key] = QUOTA_LIMIT
            get_next_api_key()
        else:
            quota_usage[current_key] += 1  # Increment quota usage
            return response.json()

@app.get("/youtube/search")
def search_youtube(q: str, max_results: int = 5):
    url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "q": q,
        "part": "snippet",
        "maxResults": max_results,
        "type": "video"
    }

    try:
        data = make_youtube_api_request(url, params)
        return data
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Function to run the FastAPI server
def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run the FastAPI server in a separate thread
import threading
thread = threading.Thread(target=run_fastapi)
thread.start()

print("FastAPI server is running on http://127.0.0.1:8000")

FastAPI server is running on http://127.0.0.1:8000


INFO:     Started server process [31051]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:45158 - "GET /youtube/search?q=python&max_results=2 HTTP/1.1" 200 OK
